
**VLAN Hopping**
**IOC 15 – VLAN Hopping: Introduction**
Technical Explanation 
VLAN hopping is an attack technique where an adversary escapes the logical boundaries of their assigned Virtual LAN (VLAN) and gains access to traffic in other VLANs. This undermines segmentation and allows the attacker to interact with systems they should be isolated from.

One of the most common forms of VLAN hopping uses double-tagged 802.1Q packets. Here’s how it works:

802.1Q VLAN Tagging:
Ethernet frames on VLAN-aware networks carry a special 4-byte tag inserted by switches to identify which VLAN the frame belongs to. This is known as IEEE 802.1Q tagging.

Double Tagging Attack:

The attacker crafts a packet with two VLAN tags (outer and inner).

The outer tag matches the native VLAN of the trunk port it reaches.
(The native VLAN is the one that a trunk port will not tag or inspect—it's assumed to be trusted.)

The first switch receives this frame and strips the outer tag, thinking it’s normal traffic from its native VLAN.

What remains is the inner tag, now exposed and trusted by the next switch in line—which forwards the packet into the VLAN identified by that inner tag.

Result: the attacker’s packet hops from their VLAN into a target VLAN they were never authorized to reach.

Trunk Ports and Native VLANs:

A trunk port is a switch interface that carries traffic for multiple VLANs, commonly used to link switches or connect to routers.

Trunk ports add or remove VLAN tags based on configuration.

The native VLAN is the VLAN that travels untagged across a trunk port. If a trunk port receives a packet without a tag, it assumes it belongs to the native VLAN.

Exploitation Mechanism:
VLAN hopping exploits misconfigurations, such as:

A trunk port that accepts untagged traffic (a misconfigured native VLAN).

Matching native VLANs on both switches, which causes one to strip the tag and the other to trust the inner tag.

Softened Interpretation / Analogy
Imagine you live in a high-rise apartment building where each floor is secured and isolated—you need a special key to reach any floor but your own. The elevators are tag-aware, and normally stop you if you try to access a restricted floor.

But now picture this:
You trick the elevator by saying, “I belong to the public lobby floor” (your outer tag—a trusted native VLAN). The elevator accepts you and drops that label because the lobby is open access. But inside your pocket, you have another forged ID badge (inner tag) for a floor you’re not authorized to visit—say, Floor 7 (VLAN 7). Now that the outer tag is stripped, the system sees your second badge and says, “Welcome to Floor 7,” opening access it never should.

This is VLAN hopping through double-tagged packets. You use a fake outer identity to slip through one checkpoint and expose a second identity to reach a protected segment.

Summary:
VLAN hopping = bypassing VLAN segmentation via crafted packets

Double-tagging = attacker inserts two VLAN tags: outer = native VLAN (trusted), inner = target VLAN

Trunk port = switch interface carrying multiple VLANs, vulnerable if it allows native VLAN traffic untagged

Native VLAN exploit = outer tag is removed at first switch, inner tag passed along to unauthorized destination



**1. Attacker Analogy**
Technical Description:
Imagine a hotel with different floors (VLANs), each requiring a special keycard for access. The attacker checks into a room on one floor but wants to spy on guests in restricted areas. By crafting a fake keycard that contains two access codes—one for their current floor (outer tag), and one for the target restricted floor (inner tag)—they trick the elevator into taking them where they shouldn’t be. This exploit relies on misconfigurations: the elevator (switch) accepts the first code as normal, but then passes the second one without proper validation.

Narrative Softening:
The attacker is like a corporate spy hiding in plain sight on the third floor of a hotel. But hidden in their luggage is a double-layered keycard. When they slide it into the elevator, it peels off the outer identity and secretly reveals the real goal: access to the executive floor. They didn’t scale the building or bust down a door—they tricked the system into treating them like they belonged somewhere they didn’t.

**2. Source of IOC (Telemetry Origin)**
Technical Description:
Alert triggered via internal switch logs forwarded to SIEM, showing abnormal double-tagged 802.1Q frames originating from a single host, bypassing VLAN segmentation boundaries.

Narrative Softening:
The front desk (SIEM) received a strange report: one guest's badge was being used to access multiple floors, and the elevator logs showed duplicate tags—an anomaly in a system that usually checks just one. Something wasn’t right about how this guest was traveling the network.

**3. Triage Framework Declaration**
Triage Type: Host-Based IOC Triage Protocol
Host-based triage is used because the initial signal originates from a system under attacker control that is actively crafting and sending anomalous Ethernet frames from within the LAN.

**4. Canonical Investigative Toolkit**
(1) Windows Event Logs
Check for 802.1X authentication records, interface bindings, and local NIC activity that may indicate configuration tampering.

(2) EDR Telemetry (if present)
Inspect user activity, custom packet crafting tools (e.g., Scapy, Yersinia), and abnormal process behaviors associated with Layer 2 manipulation.

(3) File System and Registry Inspection
Look for evidence of network utilities, malicious scripts, or unauthorized driver-level alterations to NIC behavior.

(4) Volatile Memory Capture
Search memory for evidence of double-tag crafting buffers, embedded VLAN identifiers, or forged Ethernet frame structures.

Softened Interpretation:
You interview the suspected intruder’s room. There are traces of elevator override tools in their bags (Scapy), maps of other floors (VLAN IDs), and customized badge writers (NIC driver mods). They’ve tampered with the room's electronics to trick the system into treating them as someone they’re not.

**5. Operating System Layer & Key Behaviors**
Relevant OS Layers:

Layer 1: Process Execution – Packet crafting utilities launched on the compromised host.

Layer 2: Startup/Persistence Infrastructure – If tools were installed with autostart registry keys or scheduled tasks.

Layer 6: Network Communication Context – Direct Ethernet-level traffic manipulation.

Softened Interpretation:
The attacker isn’t just passing through the building—they’ve installed devices in the walls, set up scheduled tasks to run sabotage tools at check-in, and redirected all calls from one room to another.

**6. Cross-Layer Interaction Pivots**
Pivots Identified:

Layer 1 → Layer 6 – Tool launches and immediately crafts outbound malicious Layer 2 traffic.

Layer 2 → Layer 3 – Bypass of segmentation logic allows cross-subnet movement.

Triage Pivot: Host → Network if detection reveals live cross-VLAN movement in real time.

Softened Interpretation:
They entered through a keyhole, but now they’re rewiring the building. Their actions started on a workstation but now echo through the hallways, connecting rooms that were never meant to meet.

**7. OSI Layer Relevance**
Primary OSI Layer:

Layer 2 – Data Link Layer (Crafted 802.1Q double-tag frames).
Secondary OSI Layers:

Layer 3 – IP Routing Implications

Layer 1 – Physical NIC Interface Behavior (broadcast mode misused).

Softened Interpretation:
This wasn’t a polite phone call—they tapped into the hotel’s wiring and made the floors speak to each other behind the manager’s back.

**8. Attacker Behavior Interpretation**
The attacker understands the segmentation barriers (VLANs) in place and is deliberately exploiting a weakness in switch configuration (native VLAN overlaps or trunk port assumptions). Their method reflects preparation, system awareness, and a goal of accessing internal systems not assigned to their current zone. This is likely reconnaissance-driven, lateral targeting behavior.

Softened Interpretation:
They’re not aimlessly snooping—they brought blueprints. They’re slipping into restricted areas with forged credentials, not because they’re curious, but because something specific is behind those locked doors.

**9. Defender Action Summary**
Isolate and inspect compromised host.

Review switch configurations for trunk port mismanagement or native VLAN overlap.

Update switch ACLs to reject double-tagged frames.

Enable VLAN tagging validation on trunk ports.

Deploy NetFlow or packet capture sensors to catch tagging anomalies.

Softened Interpretation:
You don’t just throw the guest out—you audit the elevators, change the passcodes, and ensure no more skeleton keycards are slipping through unchallenged.

**10. Attacker Strategy Notes**
This attacker is not breaching from the outside—they are operating from inside the perimeter, already within a trusted VLAN. Their attack exploits weaknesses in Layer 2 trust assumptions, particularly on switches with poorly secured trunk port behavior. VLAN hopping is often a pivot technique to reach sensitive data or administrative interfaces logically separated from the initial compromise point. Their goal is to escalate by movement—quietly and laterally—before detection is triggered.

**11. Who’s Who – Object Role Clarification**
Object/Entity	Role in IOC Scenario
Compromised Host	The attacker's foothold on the network, used to craft malicious frames
Network Interface Card (NIC)	Hardware interface on the compromised host used to inject traffic
802.1Q Double-Tagged Frame	The specially crafted Ethernet frame designed to trick switches
Switch (Layer 2 Device)	Device being fooled into reassigning traffic to unintended VLANs
VLAN	Segmented subnet being bypassed
Trunk Port	Switch port misused to accept/tag forwarded traffic improperly

# CONCLUSION: IOC 15 – VLAN Hopping (Double-Tagged 802.1Q Exploit)

This case study examines VLAN hopping—a network segmentation bypass technique where an attacker crafts specially tagged Ethernet frames to escape their assigned VLAN and access restricted segments. The attack exploits trust assumptions in Layer 2 switch configurations, especially mismanaged trunk ports and native VLAN behaviors.

 **Overview**

The attacker begins from within a trusted VLAN, using a compromised host to inject double-tagged 802.1Q packets. These packets are designed to manipulate how switches interpret VLAN tags. If misconfigurations exist (such as native VLAN overlaps or unfiltered trunk ports), the outer VLAN tag is stripped and the inner tag is forwarded, causing the frame to be delivered into a VLAN the attacker should never reach. This undermines logical segmentation and creates lateral movement opportunities.

## Key Concepts Covered

- How 802.1Q VLAN tagging works and why double-tagging can be dangerous
- The role of trunk ports and native VLAN configuration in switch behavior
- Host-based forensic triage for VLAN manipulation from inside a compromised system
- Network detection strategies for identifying crafted VLAN tag abuse
- Cross-layer pivot logic from host execution to network-level exploitation

## Investigative Structure

This IOC case study follows a structured triage methodology:

- **Primary Triage Protocol**: Host-Based IOC Triage – due to crafted frame generation from an attacker-controlled system
- **Canonical Investigative Tools**:
  - **Windows Event Logs** – interface and authentication anomalies
  - **EDR Telemetry** – detection of tools like Scapy, Yersinia, or raw NIC usage
  - **File System/Registry Inspection** – driver-level or tool-based persistence
  - **Volatile Memory Analysis** – extraction of double-tag payload buffers or crafted frame evidence
- **Operating System Layers Involved**:
  - Layer 1: Execution of packet tools
  - Layer 2: Startup or persistence mechanisms
  - Layer 6: Low-level network manipulation
- **OSI Layers Impacted**:
  - Layer 2: Direct VLAN tag abuse
  - Layer 3: Resulting cross-VLAN traffic redirection
  - Layer 1: Physical NIC misused for broadcast injection

## Tools & Techniques Observed

- **802.1Q double-tagged packet crafting**
- **Scapy** or **Yersinia** (custom packet creation frameworks)
- **Misuse of trunk ports on switches that fail to validate inner VLAN tags or enforce VLAN boundaries based on ingress port.**
- **Use of raw socket access or NIC driver modification** for Ethernet frame control

Raw socket access – enables direct Ethernet frame crafting outside of the standard TCP/IP stack, used to insert VLAN tags manually for double-tagging.

NIC driver modification or abuse – alters how the network interface handles traffic, potentially bypassing OS-level restrictions and enabling unauthorized frame injection at Layer 2.



## Defender Response Framework

- Isolate the suspected host and extract forensic evidence from volatile and persistent layers
- Review and correct switch trunk port configurations
- Ensure native VLANs are not shared or open to unauthorized interfaces
- Enable VLAN tag validation and ACLs to reject double-tagged frames
- Deploy NetFlow or packet capture systems tuned to detect 802.1Q anomalies

## Strategic Takeaways

VLAN hopping is not an initial compromise—it’s a **post-compromise pivot tactic** used by attackers to move laterally within a segmented environment. It bypasses network logic through **crafted deception at Layer 2**, exploiting trust relationships between switch ports and the traffic they forward. Understanding how attackers manipulate switch behavior using host-controlled NICs is key to detecting and stopping such lateral expansions before they escalate into privilege abuse or data access.

## Files

- `ioc15-vlan-hopping.ipynb`: Full case study with detailed narrative, technical analysis, and layered triage methodology

#


